# Modelo NLP - Twitter

In [1]:
import pandas as pd
from spacy import displacy
import spacy
import re
import mysql.connector as conn

In [2]:
from spacy.lang.es.stop_words import STOP_WORDS

#Crear pdf report

from io import BytesIO
from reportlab.pdfgen import canvas
from django.http import HttpResponse

#Automatizar email

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Cargo Modelo NLP

In [3]:
import pickle
import sys
sys.path.append('../')

from src.track_function import text_data_cleaning

In [ ]:
filename = 'modeloNLP_Twitter'

In [ ]:
modelo = pickle.load(open(filename, 'rb'))

In [ ]:
modelo.predict(['Estoy hasta los huevos'])

# Cargo Funciones

In [4]:
import sys
sys.path.append('../')

from src.track_function import *

## Creo base de datos necesarias

### Creando SQL Database

In [ ]:
db=conn.connect(host='localhost', user='root', passwd='123456', database='hate_blocker')

cursor=db.cursor()

cursor.execute('drop table if exists hate_blocker')

### Creando Tabla: Train Model

El objetivo de este tabla es guardar los siguientes tweets:

- Tweets correctos identificados como sentiment 1.

- Tweets incorrectos identificados como sentiment 0.

- Tweets correctos identificados como sentiment 0.

cursor.execute('drop table if exists train_model')

In [ ]:
tabla= '''
        create table train_model(
            
           tweet_id bigint ,      
           user bigint,
           content varchar(500),
           date varchar(500),
           lang varchar(500),
           sentiment bigint
            
        );
'''

cursor.execute (tabla)

### Creando Tabla: Listado Bullies

cursor.execute('drop table if exists listado_bullies')

In [ ]:
tabla= '''
        create table listado_bullies(
            
           
           tweet_id bigint,
           date varchar(500),      
           account varchar(500),
           tweet_text varchar(500),
           URL_account varchar(500),
           URL_tweet varchar(500)
            
        );
'''


cursor.execute (tabla)

# Funcion PDF

In [ ]:
def drawMyRuler (pdf): #Plantilla
    
    pdf.drawString(100, 810, 'x100')
    pdf.drawString(200, 810, 'x200')
    pdf.drawString(300, 810, 'x300')
    pdf.drawString(400, 810, 'x400')
    pdf.drawString(500, 810, 'x500')
    
    pdf.drawString(10, 100, 'y100')
    pdf.drawString(10, 200, 'y200')
    pdf.drawString(10, 300, 'y300')
    pdf.drawString(10, 400, 'y400')
    pdf.drawString(10, 500, 'y500')
    pdf.drawString(10, 600, 'y600')
    pdf.drawString(10, 700, 'y700')
    pdf.drawString(10, 800, 'y800')

# Funcion PDF

In [ ]:
def pdf(usuario, texto):
            
        #name and create pdf

        c = canvas.Canvas('hate_.pdf')
        
        #set background color (this color is yellow I do not recommend)

        c.setFillColorRGB(0,0,0)

        #Select font and font size

        c.setFont('Helvetica', 12)

        #create two text strings and set there position on the page

        c.drawString(5, 660, 'Hemos bloqueado la cuenta' + usuario + 'por el siguiente tweet:' + texto)

        #show page and save it

        c.showPage()
        c.save()
          

# Función-Modelo

In [5]:
import tweepy

## Funcion de autentificacion

In [ ]:
# 4 authentication chains

consumer_key='Qbxaw0OrPeZSFljKEp6SHI19M'
consumer_secret='kva4UfB6cK2Z6P70MGV8LAM0MMOu3SAHjuUl5t7bWxpegkCHGl'
access_key='902474996-eLIpuPP1uqYZoTWtprZuiL2FmehUURAZHT8ZBReU'
access_secret='CkVPZlMz5Lk4EMYDZzmNM2zJQHGE0ze6CA0auHqnYtEhU'

# authorize twitter, initialize tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)


def insert(sentiment):
    
    insert_query="insert into {} ({}) values {};"\
                        .format('train_model', ','.join(['tweet_id', 'user', 'content', 'date', 'lang', 'sentiment']), tuple([tweet.id_str, i.id,tweet.text, str(tweet.created_at), 'es', sentiment]))


    cursor.execute(insert_query)

    db.commit()

    return

def bullies_to_sql():
    
    insert_query="insert into {} ({}) values {};"\
                        .format('listado_bullies', ','.join(['tweet_id','date', 'account', 'tweet_text', 'URL_account','URL_tweet']), 
                                tuple([tweet.id_str, str(tweet.created_at), i.screen_name,tweet.text, 'https://twitter.com/' + i.screen_name, 'https://twitter.com/' + i.screen_name + '/status/' + tweet.id_str]))


    cursor.execute(insert_query)

    db.commit()

    return

In [ ]:
def valoracion_block():

    users = []
    users_url = []
    tweets_url = []
    usuario = []
    texto = []


    mentions = api.mentions_timeline(tweet_mode= 'extented')

    for tweet in mentions: # Entran los tweets


        users.append(tweet.user)

        for i in users:

            print(' ')

        valoracion = clf.predict([tweet.text]) 

        if valoracion == 1: # Es positivo el tweet?

            print ('No bloqueo al usuario. Guardado para entrenar.')

            print (' ')

            insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.


        elif valoracion == 0: # Es negativo el tweet?

            print('Voy a bloquear a ', tweet.user.screen_name, 'por el siguiente tweet:', tweet.text )

            verif = input ('¿Deseas bloquear a este usuario?') # Estas seguro que quieres bloquear?

            print (verif)

            if verif == 'Si': # Si

                api.create_block(tweet.user.screen_name)

                insert(0) # Metelo en sql los tweets con etiqueta 0 para entrenar.

                bullies_to_sql() # Metelo en el listado de bullies para enseñarlo al cliente.

                print('Bloqueado. Guardado para entrenar.')

                print('Guardado para enviar a cliente. ')

                usuario.append(i.name)

                texto.append(tweet.text)

                #pdf(usuario, texto)



            elif verif == 'No': # No

                insert(1) # Mete en sql los tweets con etiqueta 1 para entrenar.

                print ('Usuario no bloqueado. Guardado para entrenar')

            elif verif == 'Para': # Para

                break

In [ ]:
def mail():
    
    # assign key email aspects to variables for easier future editing
    subject = "Hater Report"
    body = 'Hola, te envío el resumen de los usuarios bloqueados: \n'
    for i in range (len(usuario)):

        body += f"- He bloqueado al usuario *{usuario[i]}* por el tweet: {texto[i]} \n" 

    sender_email = "ironhack.trabajo.final@gmail.com"
    receiver_email = "alexmendezsomoza@gmail.com"
    #file = "automate_report.pdf" # in the same directory as script
    password = "Ironhack2021"

    # Create the email head (sender, receiver, and subject)

    email = MIMEMultipart()
    email["From"] = sender_email
    email["To"] = receiver_email 
    email["Subject"] = subject

    # Add body and attachment to email

    email.attach(MIMEText(body, "plain"))

    #attach_file = open(file, "rb") # open the file
    #report = MIMEBase("application", "octate-stream")
    #report.set_payload((attach_file).read())
    #encoders.encode_base64(report)

    #add report header with the file name

    #report.add_header("Content-Decomposition", "attachment", filename = file)
    #email.attach(report)

    #Create SMTP session for sending the mail

    session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
    session.starttls() #enable security
    session.login(sender_email, password) #login with mail_id and password
    text = email.as_string()
    session.sendmail(sender_email, receiver_email, text)
    session.quit()
    
    return('Mail Sent')

In [ ]:
import sys
sys.path.append('../')

from src.track_function import text_data_cleaning

In [ ]:
text_data_cleaning('Estamos bailando en la fiesta. Alguien viene...')

In [6]:
valoracion_block()

 
Voy a bloquear a  BryanGilMVP por el siguiente tweet: @xinxinricardo wow
¿Deseas bloquear a este usuario?Si
Si
Bloqueado. Guardado para entrenar.
Guardado para enviar a cliente. 
 
 
Voy a bloquear a  CarlosLarios96 por el siguiente tweet: @xinxinricardo Pf
¿Deseas bloquear a este usuario?No
No
Usuario no bloqueado. Guardado para entrenar
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
Voy a bloquear a  BryanGilMVP por el siguiente tweet: @xinxinricardo DAME UN MES!!!!!
¿Deseas bloquear a este usuario?Si
Si
Bloqueado. Guardado para entrenar.
Guardado para enviar a cliente. 
 
 
 
 
 
Voy a bloquear a  alexseun874 por el siguiente tweet: @xinxinricardo @atletienglish Congratulations! You did it.
¿Deseas bloquear a este usuario?Si
Si
Bloqueado. Guardado para entrenar.
Guardado para enviar a cliente. 
 
 
 
 
 
 
Voy a bloquear a  alexseun874 por el siguiente tweet: @xinxinricardo @atletienglish 💪💪💙💙
¿Deseas bloquear a este usuario?Parar
Parar
 
 
 
 
 
 
 
Voy a bloquear

(['Bryan the Insect 🦟',
  'Bryan the Insect 🦟',
  'Uncle Sam - EndPoliceBrutality'],
 ['@xinxinricardo wow',
  '@xinxinricardo DAME UN MES!!!!!',
  '@xinxinricardo @atletienglish Congratulations! You did it.'])

In [7]:
usuario, texto = valoracion_block()

 
Voy a bloquear a  CarlosLarios96 por el siguiente tweet: @xinxinricardo Pf
¿Deseas bloquear a este usuario?Si
Si
Bloqueado. Guardado para entrenar.
Guardado para enviar a cliente. 
 
 
Voy a bloquear a  pedro_pescado por el siguiente tweet: @xinxinricardo https://t.co/uOyinG7VOz
¿Deseas bloquear a este usuario?Si
Si
Bloqueado. Guardado para entrenar.
Guardado para enviar a cliente. 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
 
 
 
 
No bloqueo al usuario. Guardado para entrenar.
 
 
 
 
 
 
 
 
 
 
 
Voy a bloquear a  Nicolas14859969 por el siguiente tweet: @elanonimoboss @ZizouBiwenger @aangelrp @biwenger @FarolilloTeam @PicaPlomo @futbol_fantasy @JornadaPerfect

['@xinxinricardo Pf',
 '@xinxinricardo https://t.co/uOyinG7VOz',
 '@elanonimoboss @ZizouBiwenger @aangelrp @biwenger @FarolilloTeam @PicaPlomo @futbol_fantasy @JornadaPerfecta @GuruBiwenger @DudasBiwenger 3']

In [10]:
mail(usuario,texto)

NameError: name 'MIMEMultipart' is not defined

In [12]:
modelo()

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function text_data_cleaning at 0x7fd11ed3b940>)),
                ('clf', LinearSVC())])